# M3. Actividad
Los agentes involucrados en este modelo serian los automoviles y los semaforos. 
Los carros guardaran informacion en su clase Vehicle de la posicion en la que se encuentran en el grid, el codigo del color del carro, la direccion en la que se mueve el carro, ya sea de arriba hacia abajo o de izquierda a derecha.
La clase semaforo basicamente guarda la posicion del semaforo en el grid, el estado del semaforo 2 siendo rojo, 1 siendo amarillo y 0 siendo verde, aparte guarda el codigo del color actual del semaforo. El semaforo tiene una funcion propia la cual en base al numero de carros antes del semaforo es al cual le va a dar preferencia.
El tercer agente que seria Grass no sirve de nada y basicamente solo sirve para pintar el mapa mas bonito en el plot.
En el modelo lo que hace es avanzar un carro siempre y cuando no tenga un carro en la posiciones depues de el, tambien hace que si el carro tiene un semaforo a un lado y esta en rojo el carro se parara y no avanzara hasta el color verde. En caso de no existir carros desde la posicion 1 hasta la posicion (tgrid/2)-1 esto aplicado tanto en x como en y entonces el semaforo se pondra en amarillo.


In [1]:
import numpy as np
import agentpy as ap
import matplotlib.pyplot as plt
import IPython

class Vehicle(ap.Agent):
    def setup(self):
        self.tGrid = self.p['Grid']
        self.grid = self.model.grid
        self.pos = [0, 0]
        self.road = 1
        self.side = [1, 0]
        self.speed = 1
        self.crossed = False

    def direction(self):
        self.pos = self.grid.positions[self]
        if self.pos[1] == 0:
            self.side = [0, 1]

    def movement(self):
        self.direction()
        return (self.speed * self.side[0], self.speed * self.side[1])

    def route(self):
        self.pos = self.grid.positions[self]
        if self.pos[1] == 0:
            return 'HORIZONTAL'
        return 'VERTICAL'

class StopSign(ap.Agent):
    def setup(self):
        self.tGrid = self.p['Grid']
        self.status = 1
        self.road = 3
        self.grid = self.model.grid
        self.pos = [0, 0]

    def positions(self):
        self.pos = self.grid.positions[self]

    def change_state(self):
        self.positions()
        self.status = 2
        self.road=2
        if self.model.n_cars_1 > self.model.n_cars_2:
            if self.pos[0] == int((self.tGrid/2)+1):
                self.status = 0
                self.road=4
        elif self.model.n_cars_1 < self.model.n_cars_2:
            if self.pos[1] == int((self.tGrid/2)+1):
                self.status = 0
                self.road=4
        else:
            if self.pos[1] == int((self.tGrid/2)+1):
                self.status = 0
                self.road=4

class Grass(ap.Agent):
    def setup(self):
        self.road = 5
        self.condition = 1

class IntersectionModel(ap.Model):
    def setup(self):
        self.con=0
        self.tGrid = self.p['Grid']
        self.grid = ap.Grid(self, [self.tGrid] * 2, torus=True,track_empty=True)
        self.n_cars_1 = 0
        self.n_cars_2 = 0

        n_vehicles = self.p['Vehicles']
        n_roads = (self.tGrid*self.tGrid)-(self.tGrid*2)-1

        self.vehicles = ap.AgentList(self, n_vehicles, Vehicle)
        self.road = ap.AgentList(self, n_roads, Grass)
        self.stop_sign = ap.AgentList(self, 2, StopSign)
        self.vehicles.grid = self.grid

        grass_pos = []
        for i in range(self.tGrid):
            for j in range(self.tGrid):
                if i != (self.tGrid/2) and j != (self.tGrid/2):
                    if i != (self.tGrid/2)+1 or j != (self.tGrid/2)-1:
                        if j != (self.tGrid/2)+1 or i != (self.tGrid/2)-1:
                            grass_pos.append((i,j))

            
        self.grid.add_agents(self.stop_sign, positions=[(int((self.tGrid/2)-1), int((self.tGrid/2)+1)), (int((self.tGrid/2)+1), int((self.tGrid/2)-1))])
        self.grid.add_agents(self.road, grass_pos)
        
        vehicles_positions=[]
        positions=[(0, int(self.tGrid/2)), (int(self.tGrid/2), 0)]
            
        for i in range (n_vehicles):
            rand_pos = np.random.randint(0, 2)
            vehicles_positions.append(positions[rand_pos])

        new_car=ap.AgentList(self,1,Vehicle)
        self.grid.add_agents(self.vehicles,vehicles_positions)

    def step(self):
        movimiento=True
        state=False
        moving_cars_1 = self.vehicles
        for car in moving_cars_1:
            agent_pos=self.grid.positions[car]
            for i in range(1,int((self.tGrid/2))):
                if (int(self.tGrid/2) == agent_pos[0])and(int(i) == agent_pos[1]):
                    self.n_cars_1 += 1
                    state=True
                if  (int(self.tGrid/2) == agent_pos[1])and(int(i) == agent_pos[0]):
                    self.n_cars_2 += 1
                    state=True
        if state:
            stop_light = self.stop_sign
            for semaforo in stop_light:    
                if state:
                    semaforo.change_state()
                else:
                    semaforo.status=1
                    semaforo.road=3
        '''
        for car in moving_cars_1:
            
        '''
       
        for agent in self.grid.agents:
            agent_pos=self.grid.positions[agent]
            movimiento=True
            if agent.type == 'Vehicle':
                for neighbor in self.grid.neighbors(agent):
                    if agent.route() == 'VERTICAL':
                        if self.grid.positions[neighbor][1]==agent_pos[1]+1 and self.grid.positions[neighbor][0]==agent_pos[0]:
                            if   neighbor.type=='Vehicle':    
                                movimiento=False
                                break
                        if self.grid.positions[neighbor][1]==agent_pos[1] and self.grid.positions[neighbor][0]==agent_pos[0]+1:
                            if  neighbor.type=='StopSign':
                                if neighbor.status==2:
                                    movimiento=False
                                    break
                                else:
                                    self.n_cars_1-=1
                                    break
                        
                    if agent.route() == 'VERTICAL':
                        if self.grid.positions[neighbor][0]==agent_pos[0]+1 and self.grid.positions[neighbor][1]==agent_pos[1]:
                            if  neighbor.type=='Vehicle':
                                movimiento=False
                                break
                        if self.grid.positions[neighbor][0]==agent_pos[0] and self.grid.positions[neighbor][1]==agent_pos[1]+1:
                            if  neighbor.type=='StopSign':
                                if neighbor.status==2:
                                    movimiento=False
                                    break
                                else:
                                    self.n_cars_2-=1
                                    break
                if movimiento:
                    coordinates_move=agent.movement()
                    self.grid.move_by(agent,coordinates_move)

    def end(self):
        pass


parameters = {
    'Vehicles': 15,
    'steps': 100,
    'Grid':20,
}


def animation_plot(model, ax):
    attr_grid = model.grid.attr_grid('road')
    #0 es gris claro, 1 es azul, 2 es rojo, 3 amarillo, 4 verde fuerte, 5 es verde claro
    color_dict = {0: '#d5e5d5', 1: '#0000ff', 2: '#d62c2c', 3: '#FFFF00', 4: '#21d41e', 5:'#7FC97F', None: '#d5e5d5'}  
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)


fig, ax = plt.subplots()
model = IntersectionModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=30))